In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load libraries

In [ ]:
import pandas as pd
import nltk
import re

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import nltk
import matplotlib.pyplot as plt
import numpy as np
import re

import string
from nltk.corpus import stopwords
from nltk import FreqDist

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from pandas_datareader import data as pdr
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas_datareader as pdr
import datetime as dt
from datetime import datetime, date
import yfinance as yf
import xgboost as xgb
yf.pdr_override()
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as sm
from pandas_datareader import data as pdr
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import SimpleExpSmoothing
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display
import holidays
import plotly.express as px

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import joblib

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_lg")

# Load functions

In [ ]:
def clean_text(columna_datos):
    pattern = r'''(?x)                 # set flag to allow verbose regexps
              (?:[A-Z]\.)+         # abbreviations, e.g. U.S.A.
              | \w+(?:-\w+)*       # words with optional internal hyphens
              | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
              | \.\.\.             # ellipsis
              | [][.,;"'?():-_`]   # these are separate tokens; includes ], [
    '''
    textos = columna_datos

    # Tokenizar cada fila
    tokens = [nltk.regexp_tokenize(texto, pattern) for texto in textos]

    columna_tokens = pd.DataFrame()
    columna_tokens["Tokens"] = tokens

    # Lematizador
    lemmatizer = WordNetLemmatizer()
    columna_tokens["Tokens"] = columna_tokens["Tokens"].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

    # Eliminar stopwords:
    columna_tokens["Tokens"] = columna_tokens["Tokens"].apply(lambda tokens: [token for token in tokens if token.lower() not in stop_words])

    # Eliminar carácteres especiales y palabras cortas:
    columna_tokens["Tokens"] = columna_tokens["Tokens"].apply(lambda tokens: [token for token in tokens if token.isalnum() and len(token) >=3])

    # Convertir todo a minúsculas:
    columna_tokens["Tokens"] = columna_tokens["Tokens"].apply(lambda tokens: [token.lower() for token in tokens])

    # Convertir todo a texto:
    columna_tokens["Tokens"] = columna_tokens["Tokens"].apply(lambda tokens: ' '.join([token.lower() for token in tokens]))

    return columna_tokens

# Load news data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mercados financieros/Bases de datos/Noticias AAPL Market Watch.csv")

In [ ]:
data["text"] = clean_text(data["Titulo"])

In [ ]:
#data["Titulo"] = data['Titulo'].astype(str)

In [ ]:
#data = data.dropna()

> Adjust news according to trading time

In [ ]:
data['Fecha'] = pd.to_datetime(data['Fecha'])

data['Hora'] = pd.to_timedelta(data['Hora'])

data['Fecha'] = data['Fecha'] + pd.to_timedelta((data['Hora'] > '15:00:00').astype(int), unit='D')

data = data = data.drop(columns=["Hora"])

data = data.sort_values(by="Fecha")

In [ ]:
data

,Etiqueta,Titulo,Fecha,Enlace,Autor,text
4279,Barron's,Can’t Find that New iPhone? You’re Not Alone,2016-10-19,https://www.marketwatch.com/articles/cant-find...,Barron's,find new iphone alone
539,Barron's,Why Apple Pay Is Way More Secure Than a Credit...,2016-10-20,https://www.marketwatch.com/articles/why-apple...,Barron's,apple pay way secure credit card
3219,Barron's,"Apple: Sales of iPhone, iPad and Mac All Shrin...",2016-10-26,https://www.marketwatch.com/articles/apple-sal...,Barron's,apple sales iphone ipad mac shrink better sign...
4278,Barron's,Apple’s New MacBooks Leave Us Wanting More,2016-10-28,https://www.marketwatch.com/articles/apples-ne...,Barron's,apple new macbooks leave wanting
1099,Barron's,Could Apple Buy Time Warner? Would It? Should It?,2016-10-31,https://www.marketwatch.com/articles/could-app...,Barron's,could apple buy time warner would
...,...,...,...,...,...,...
4,Opinion,3 reasons it’s safe to own the ‘Magnificent Se...,2023-12-04,https://www.marketwatch.com/story/3-reasons-wh...,Michael Brush,reason safe magnificent seven stock
3,Sin etiqueta,"Apple Inc. stock falls Monday, underperforms m...",2023-12-05,https://www.marketwatch.com/data-news/apple-in...,MarketWatch Automation,apple inc stock fall monday underperforms market
2,Taxes,Supreme Court Case Puts Focus on Wealth,2023-12-05,https://www.marketwatch.com/articles/supreme-c...,Barron's,supreme court case puts focus wealth
1,Sin etiqueta,Here’s what’s worth streaming in December 2023...,2023-12-05,https://www.marketwatch.com/story/heres-whats-...,Mike Murphy,worth streaming december 2023 much new yet sti...


> Grouping data by each date

In [ ]:
# Droping link, close, label and autor columns to simplify the problem
data = data.drop(columns=["Enlace","Etiqueta","Autor"])

In [ ]:
data = data.groupby('Fecha')['text'].agg(lambda x: ' '.join(x)).reset_index()
data.tail(5)

,Fecha,text
1858,2023-11-30,apple could find new card partner means custom...
1859,2023-12-01,apple stock tough decembers time could differe...
1860,2023-12-02,say corporate bond dull apple bond seen return...
1861,2023-12-04,beyoncé rule box office concert movie earns mi...
1862,2023-12-05,apple inc stock fall monday underperforms mark...


In [ ]:
# Weekly
data = data.groupby(pd.Grouper(key='Fecha', freq='W'))['text'].agg(lambda x: ' '.join(x)).reset_index()
data

,Fecha,text
0,2016-10-23,find new iphone alone apple pay way secure cre...
1,2016-10-30,apple sales iphone ipad mac shrink better sign...
2,2016-11-06,could apple buy time warner would starbucks ea...
3,2016-11-13,
4,2016-11-20,samsung apple could really change cars apple i...
...,...,...
368,2023-11-12,apple nvidia rest big tech winners time 500 49...
369,2023-11-19,superhero fatigue marvels flop opening weekend...
370,2023-11-26,arm licensing deals power growth stock buy ana...
371,2023-12-03,hunger games prequel hold wish napoleon conque...


# Load financial data

In [ ]:
# Convertir la columna 'Fecha' a formato de fecha
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Ordenar el DataFrame por la columna 'Fecha' de manera ascendente
data = data.sort_values(by='Fecha')

# Organizar bien los data a partir de la fecha final de scraping
data = data[data['Fecha'] >= '2018-08-07']

In [ ]:
from datetime import datetime, date

# Input fechas:
start_dt = "2018-08-07"
end_dt = "2023-12-06"

# Variables fechas:
start_date = datetime.strptime(start_dt, "%Y-%m-%d")
end_date = datetime.strptime(end_dt, "%Y-%m-%d")

# Fecha hasta hoy:
today = date.today()

# Obtener price de los tickers:
tickers = ["AAPL"]
asset = "AAPL"
price = {}

# Obtener los price para cada ticker
for ticker in tickers:
    price[ticker] = yf.download(ticker, start=start_date, end=end_dt)
    price[ticker].index = pd.to_datetime(price[ticker].index)


price = price[ticker]
price = price["Close"]
price = pd.DataFrame(price)

# Marco de tiempo semanal:
price = price.resample("W").last()

# Cambio del price:
price['Cambio'] = (price['Close'] > price['Close'].shift()).astype(int)

# Columna fecha:
price['Fecha'] = pd.to_datetime(price.index)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Fusionar los DataFrames utilizando la columna 'Fecha'
data = pd.merge(price, data, on='Fecha', how='left')

In [ ]:
# Crear la columna de 1 lag de titulo
#data['Titulo'] = data['Titulo'].shift(2)
#data = data.dropna()

In [ ]:
data = data.drop(columns=["Close"])

# Processing text with spacy word vectors

In [ ]:
data['text'] = data['text'].astype(str)

In [ ]:
data['vector'] = data['text'].apply(lambda text: nlp(text).vector)

In [ ]:
# save vectorized data
#data.to_csv("/content/drive/MyDrive/Colab Notebooks/Mercados financieros/Proyectos/Análisis de noticias/aaplnews_dataprocessed.csv", index=False)

In [ ]:
data.head(3)

,Cambio,Fecha,text,vector
0,0,2018-08-12,apple worth trillion tim cook warren buffett g...,"[-0.30256638, 0.13565062, -1.6081395, 1.332029..."
1,1,2018-08-19,foxconn report surprise profit decline amazon ...,"[-0.39143956, 0.015913634, -1.8215661, 1.51371..."
2,0,2018-08-26,dorsey say twitter arbiter truth ban user view...,"[-0.27154863, 0.47968432, -1.5685841, 0.722241..."


In [ ]:
data.Cambio.value_counts()

1    161
0    118
Name: Cambio, dtype: int64

# Preparing data for models

In [ ]:
#data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Mercados financieros/Proyectos/Análisis de noticias/aaplnews_dataprocessed.csv")

In [ ]:
data = data.drop(columns=["text"])

In [ ]:
data["vector"] = data["vector"].shift(1)
data = data.dropna()
data

,Cambio,Fecha,vector
1,1,2018-08-19,"[-0.30256638, 0.13565062, -1.6081395, 1.332029..."
2,0,2018-08-26,"[-0.39143956, 0.015913634, -1.8215661, 1.51371..."
3,1,2018-09-02,"[-0.27154863, 0.47968432, -1.5685841, 0.722241..."
4,0,2018-09-09,"[-0.34183934, 0.2000844, -1.4231125, 1.0632282..."
5,1,2018-09-16,"[-0.3980926, 0.4289265, -1.840185, 1.4532661, ..."
...,...,...,...
274,1,2023-11-12,"[-0.317088, 0.273185, -2.0725846, 1.110082, 1...."
275,1,2023-11-19,"[-0.08808952, 0.5629203, -2.095874, 0.88600934..."
276,1,2023-11-26,"[-0.06630588, 0.37011585, -1.6944733, 1.082822..."
277,1,2023-12-03,"[-0.53039324, -0.19918923, -1.7696289, 1.40139..."


In [ ]:
data = data.set_index("Fecha")

# Building classification models

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.vector.sort_index().values,
    data.Cambio.sort_index(),
    test_size=0.3,
    random_state=42,
    stratify=data.Cambio
)

In [ ]:
# Stack vectors for getting 2 dims numpy arrays
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

# Scale data to convert negative values into a positive scale
scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

# Definir una lista de clasificadores
classifiers = [
    MultinomialNB(),
    KNeighborsClassifier(),
    GradientBoostingClassifier(),
    XGBClassifier()
]

# Test models

In [ ]:
# Iterar sobre cada clasificador
for clf in classifiers:
    # Imprimir el nombre del clasificador
    print(f"{'='*64}\n{clf.__class__.__name__}\n{'='*64}")

    # Ajustar el modelo
    clf.fit(scaled_train_embed, y_train)

    # Hacer predicciones
    y_pred = clf.predict(scaled_test_embed)

    # Imprimir el informe de clasificación
    print(accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

MultinomialNB
0.6428571428571429
              precision    recall  f1-score   support

           0       0.73      0.23      0.35        35
           1       0.63      0.94      0.75        49

    accuracy                           0.64        84
   macro avg       0.68      0.58      0.55        84
weighted avg       0.67      0.64      0.58        84

KNeighborsClassifier
0.5476190476190477
              precision    recall  f1-score   support

           0       0.46      0.51      0.49        35
           1       0.62      0.57      0.60        49

    accuracy                           0.55        84
   macro avg       0.54      0.54      0.54        84
weighted avg       0.56      0.55      0.55        84

GradientBoostingClassifier
0.6071428571428571
              precision    recall  f1-score   support

           0       0.55      0.34      0.42        35
           1       0.63      0.80      0.70        49

    accuracy                           0.61        84
   macro 

> Historical Accuracy chart

In [ ]:
y_test = pd.DataFrame(y_test.sort_index())

In [ ]:
y_pred = pd.DataFrame(data={'Cambio': y_pred}, index=y_test.index)
y_pred = y_pred.sort_index()

In [ ]:
y_test['Exactitud'] = 0
y_test['Exactitud'] = y_test['Cambio'].eq(y_pred['Cambio']).astype(int)

# Sumar 1 si la predicción fue correcta, restar -1 si fue incorrecta
y_test['Exactitud'] = y_test['Exactitud'].replace({1: 1, 0: -1}).cumsum()

In [ ]:
fig = px.line(y_test, x=y_test.index, y=['Exactitud'], title='Gráfico interactivo de Serie de Tiempo',
              labels={'value': 'Valor', 'variable': 'Variable', 'index': 'Fecha'},
              line_shape='linear')

fig.show()